In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

**What Are Tools in LangChain?**

Tools in LangChain are external functionalities that an LLM can leverage during its workflow. These functionalities are defined as actions or tasks that extend the model's capabilities beyond simple text generation. 

**Key Features of LangChain Tools**

* Dynamic Integration: Tools enable LLMs to interact with real-world systems, such as APIs or databases, in real-time.  
* Modularity: You can add or remove tools easily, creating flexible and reusable workflows.  
* Extensibility: Custom tools can be defined to handle specific use cases, such as fetching data from a proprietary system or performing domain-specific calculations.  

**Common Use Cases** 
*   API Integration: Calling weather APIs, stock market data, or translation services.  
*   Database Queries: Retrieving records from SQL or NoSQL databases.  
*   Custom Scripts: Executing Python functions or scripts to process data dynamically.



![image.png](attachment:image.png)

### Integrating an External API as a Tool

In [0]:
import requests

# Define a custom tool function for fetching weather data
def fetch_weather(location):
    api_key = "dd4d970d06abc34ddac9c3cb2f5d4546"
    url = "http://api.openweathermap.org/data/2.5/weather?q=" + location + "&appid=" + api_key
    try:
        response = requests.get(url)
        data = response.json()
        if response.status_code == 200:
            weather = data["weather"][0]["description"]
            temperature = data["main"]["temp"]
            return f"The weather in {location} is {weather} with a temperature of {temperature} kelvin."
        else:
            return f"Error: {data['message']}"
    except Exception as e:
        return f"Error: {e}"

In [0]:
fetch_weather("kolkata")

'The weather in kolkata is haze with a temperature of 292.12 kelvin.'

In [0]:
# Define the Tool in LangChain

from langchain.tools import Tool

# Define Weather Tool
weather_tool = Tool(
    name = "WeatherTool",
    func = fetch_weather,
    description= "A tool for fetching weather data of a location")



In [0]:
# Create an Agent to use the Tool

from langchain.agents import initialize_agent,AgentType
from langchain.llms import OpenAI

llm = OpenAI(model = 'gpt-3.5-turbo-instruct',temperature=0.7)

# Create the Agents
weather_agent = initialize_agent(
    name = "WeatherAgent",
    agents = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools = [weather_tool],
    llm = llm,
    verbose = True
)

In [0]:
query = "What is the weather in Kolkata?"

response = weather_agent.run(query)

print(response)



> Entering new WeatherAgent chain...
 I should use the WeatherTool to get the weather data for Kolkata
Action: WeatherTool
Action Input: Kolkata
Observation: The weather in Kolkata is haze with a temperature of 292.12 kelvin.
Thought: I now know the final answer
Final Answer: The weather in Kolkata is haze with a temperature of 292.12 kelvin.

> Finished chain.
The weather in Kolkata is haze with a temperature of 292.12 kelvin.


### Enhance the above chain by adding TimeZone Tool to fetch timezone information

* Create a new function for fetching time zone data using an API like Google Maps or WorldTimeAPI.  
* Wrap the function in a Tool object.  
* Add the new tool to the agent and test the workflow.



In [0]:
# Define a Custom Tool for fetching time zone data

def fetch_time_zone(location):
    try:
        time_zones = {
            "Kolkata": "Asia/Kolkata",
            "New York": "America/New_York",
            "London": "Europe/London",
            "Tokyo": "Asia/Tokyo",
            "Sydney": "Australia/Sydney",
            "Paris": "Europe/Paris",
        }
        return f"The time zone of {location} is {time_zones.get(location,'Unknown')}"
    except Exception as e:
        return f"Error: {e}"
    

In [0]:
# Define Time Zone Tool

time_zone_tool = Tool(
    name = "TimeZoneTool",
    func = fetch_time_zone,
    description = "A tool for fetching time zone data of a location"
)

In [0]:
# Create the Agent

agent = initialize_agent(
    tools = [weather_tool,time_zone_tool],
    llm = llm,
    agents = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

In [0]:
# Test the enhanced workflow

query = "Tell Me the Weather and TimeZone of Kolkata"
response = agent.run(query)
print(response)



> Entering new AgentExecutor chain...
 I should use both the WeatherTool and TimeZoneTool to retrieve the necessary information
Action: WeatherTool
Action Input: Kolkata
Observation: The weather in Kolkata is haze with a temperature of 292.12 kelvin.
Thought: Now I need to gather the timezone information for Kolkata
Action: TimeZoneTool
Action Input: Kolkata
Observation: The time zone of Kolkata is Asia/Kolkata
Thought: I now know the final answer
Final Answer: The weather in Kolkata is haze with a temperature of 292.12 kelvin and the time zone is Asia/Kolkata.

> Finished chain.
The weather in Kolkata is haze with a temperature of 292.12 kelvin and the time zone is Asia/Kolkata.


### Building a Multi Step Chain with Tools and Memory

**Create a LangChain workflow that** 

* Accepts user input dynamically.  
* Uses tools for fetching external data.  
* Retains conversation context using memory.  
* Combines multiple steps to deliver a coherent response

In [0]:
from langchain.chains import LLMChain,SequentialChain,ConversationChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.agents import AgentType
import requests

In [0]:
memory = ConversationBufferMemory()

weather_prompt = PromptTemplate(
    input_variables = ["location"],
    template = "What is the weather in {location}?")
time_zone_prompt = PromptTemplate(
    input_variables = ["location"],
    template = "Fetch the timezone for the {location}?")

summary_prompt = PromptTemplate(
    input_variables=['weather_info', 'time_zone_info'],
    template = "Provide a Summary: {weather_info} and {time_zone_info}")



In [0]:
# Wrap tools in LLMChain
weather_chain = SequentialChain(
    llm=llm,
    chains = [weather_tool],
    prompt=weather_prompt,
    output_key='weather_info'  # Unique output key
)
time_zone_chain = LLMChain(
    llm=llm,
    prompt=time_zone_prompt,
    output_key='time_zone_info'  # Unique output key
)
summary_chain = LLMChain(
    llm=llm,
    prompt=summary_prompt,
    output_key='summary'  # Final output key
)


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[85], line 2
      1 # Wrap tools in LLMChain
----> 2 weather_chain = SequentialChain(
      3     llm=llm,
      4     chains = [weather_tool],
      5     prompt=weather_prompt,
      6     output_key='weather_info'  # Unique output key
      7 )
      8 time_zone_chain = LLMChain(
      9     llm=llm,
     10     prompt=time_zone_prompt,
     11     output_key='time_zone_info'  # Unique output key
     12 )
     13 summary_chain = LLMChain(
     14     llm=llm,
     15     prompt=summary_prompt,
     16     output_key='summary'  # Final output key
     17 )

File d:\2. My WorkSpace and Notes\1. Demystify Folders\GenerativeAI-Demystified\.venv\lib\site-packages\langchain_core\load\serializable.py:125, in Serializable.__init__(self, *args, **kwargs)
    123 def __init__(self, *args: Any, **kwargs: Any) -> None:
    124     """""

In [0]:
from langchain.chains import SequentialChain

# Combine the chains
multi_step_chain = SequentialChain(
    chains=[weather_chain, time_zone_chain, summary_chain],
    input_variables=['location'],
    output_variables=['summary'],
    verbose=True  # Enables detailed logging for debugging
)

In [0]:
# # Create Individual Chains

# weather_chain = SequentialChain(
#     chains = [weather_tool],
#     input_variables = ["location"],
#     output_variables = ["weather"]
# )

# time_zone_chain = SequentialChain(
#     chains = [time_zone_tool],
#     input_variables = ["location"],
#     output_variables = ["time_zone"]
# )

# summary_chain = SequentialChain(
#     chains = [weather_chain, time_zone_chain],
#     input_variables = ["weather", "time_zone"],
#     output_variables = ["summary"]
# )

In [0]:
response = multi_step_chain.run(location="Kolkata")



> Entering new SequentialChain chain...

> Finished chain.


In [0]:
print(response)



In summary, the weather in Kolkata, India is cloudy with a temperature of 27°C and a possibility of thunderstorms. The city follows the Asia/Kolkata timezone.


In [0]:
print(weather_chain.run(location="Kolkata"))



The current weather in Kolkata, India is 86°F (30°C) with partly cloudy skies. The high for today is 88°F (31°C) and the low is 74°F (23°C). There is a chance of thunderstorms in the evening.


In [0]:
TBD